

# Homework 2 - Recurrent Neural Networks

[link text](https://)In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.


In [20]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`. 

In [21]:
class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    # Batch size * sequence length * vocabulary length (N+1)
    self.hidden_size = hidden_size
    self.gru = torch.nn.GRU(input_size = (N+1), hidden_size = hidden_size, batch_first = True, num_layers = 2) 
    self.linear = torch.nn.Linear(hidden_size, N+1)

  def forward(self, x):
    logits, hidden = self.gru(x)
    logits = self.linear(logits)
    return (logits.permute(0, 2, 1))

  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z. 
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.
    ch_dict = {string.ascii_lowercase[i]:i+1 for i in range(26)}
    ch_dict[" "] = 0
    int_dict = {v:k for (k, v) in ch_dict.items()}
      
    s_on_hot = torch.stack([idx_to_onehot(torch.tensor(ch_dict[ch])) for ch in s])
    logits = self.forward(s_on_hot.unsqueeze(0)) #Just the inference code, removing batch size 

    logits_list = logits.argmax(1).cpu().numpy()
    output = "".join([int_dict[num] for num in logits_list[0]])
                      
    return output

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [22]:
D = DELAY
t1 = idx_to_onehot(torch.tensor(DELAY))
train_size = len(ds)
batch_size = 64
dataloader = torch.utils.data.DataLoader(ds, batch_size=batch_size)

In [23]:
hidden_size = 100
lr = 0.001
device = "cpu"
model = GRUMemory(hidden_size)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr = lr)

In [24]:
def train_model():
  model.train()
  hidden = None
  total_values_in_one_chunk = train_size * ds.seq_length
  correct = 0
  for batch in dataloader:
    # Extracting the data and the target given the batch 
    data, target = batch
    data = idx_to_onehot(data).to(device)
    target = target.to(device)

    # Training pipeline begins
    optimizer.zero_grad()
    if hidden is not None: hidden.detach_()
    logits = model(data)
    loss = criterion(logits, target)
    loss.backward()
    optimizer.step()
    pred = logits.argmax(1)

In [25]:
def test_model(model, sequence_length=15):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    for c1, c2 in zip(s[:-D], result[D:]):
      correct += int(c1 == c2)
    total += len(s) - D

  return correct / total

In [26]:
import time
import tqdm
start_time = time.time()
n_epochs = 1
for epoch in tqdm.tqdm(range(1, n_epochs + 1)):
  train_model()
end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

100%|██████████| 1/1 [01:15<00:00, 75.92s/it]


tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [204]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

In [205]:
DATASET_SIZE = 200000
ds = VariableDelayEchoDataset(size=DATASET_SIZE)

### Model

And the model.

In [206]:
# help(torch.nn.GRU)

In [222]:
class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, max_delay):
    super().__init__()
    self.hidden_size = hidden_size
    self.gru = torch.nn.GRU(2*(N+1), hidden_size, num_layers = 2, batch_first = True)
    self.linear = torch.nn.Linear(hidden_size, N+1)
    self.max_delay = max_delay


  def forward(self, x, delays):
    logits, _ = self.gru(x)
    logits = self.linear(logits)
    return (logits.permute(0, 2, 1))

  @torch.no_grad()
  def test_run(self, s, delay):
    # This function accepts one string s containing lowercase characters a-z, 
    # and a delay - the desired output delay.
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.

    ch_dict = {string.ascii_lowercase[i]:i+1 for i in range(26)}
    ch_dict[" "] = 0
    int_dict = {v:k for (k, v) in ch_dict.items()}
      
    s_on_hot = torch.stack([idx_to_onehot(torch.tensor(ch_dict[ch])) for ch in s])
    # print (s_on_hot.size())
    j2 = idx_to_onehot(torch.tensor(delay))
    # print (j2.size())
    j2 = j2.unsqueeze(0).repeat(s_on_hot.size()[0], 1)
    # print (j2.size())
    s_on_hot = torch.cat((s_on_hot, j2), dim = 1)
    # print (s_on_hot.size())


    logits = self.forward(s_on_hot.unsqueeze(0), 1) #Just the inference code, removing batch size 

    logits_list = logits.argmax(1).cpu().numpy()
    output = "".join([int_dict[num] for num in logits_list[0]])
    # output = "0"*delay + output[len(output) - delay]
                      
    return output

### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [223]:
train_size = len(ds)
batch_size = 64
dataloader = torch.utils.data.DataLoader(ds, batch_size=batch_size)
hidden_size = 100
lr = 0.001
MAX_DELAY = 8
device = "cpu"
model = VariableDelayGRUMemory(hidden_size, MAX_DELAY)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr = lr)

In [224]:
def train_model():
  model.train()
  hidden = None
  total_values_in_one_chunk = train_size * ds.seq_length
  correct = 0
  for data, delay, target  in dataloader:
    # Extracting the data and the target given the batch 
    data = idx_to_onehot(data)
    target = target.to(device)
    j1 = idx_to_onehot(torch.tensor(delay))
    j1 = j1.unsqueeze(1).repeat(1, data.size()[1], 1)
    data = torch.cat((data, j1), dim = 2).to(device) 
    # print (data.size())     

    # Training pipeline begins
    optimizer.zero_grad()
    if hidden is not None: hidden.detach_()
    logits = model(data, 1)
    loss = criterion(logits, target)
    loss.backward()
    optimizer.step()
    pred = logits.argmax(1)
    # break

In [225]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, MAX_DELAY)
    result = model.test_run(s, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d
  print ("Accuracy is ", correct/total)
  return correct / total

In [226]:
import time
start_time = time.time()

n_epochs = 5
for epoch in tqdm.tqdm(range(1, n_epochs + 1)):
  train_model()
  # break
end_time = time.time()
duration = end_time - start_time
accuracy = test_variable_delay_model(model)

end_time = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'accuracy is too low'
print('tests passed')

  0%|          | 0/5 [00:00<?, ?it/s]<ipython-input-224-658c68cf6d39>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  j1 = idx_to_onehot(torch.tensor(delay))
100%|██████████| 5/5 [08:38<00:00, 103.75s/it]


Accuracy is  0.9972697939935468
Accuracy is  0.9978905571410845
tests passed
